In [1]:
import numpy as np
import pandas as pd
import re
from numpy import argmax, array
import keras
import tensorflow
# Import various componenets for model building
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LSTM, Input, TimeDistributed, SimpleRNN
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from scipy.stats import bernoulli
# Import the backend
from keras import backend as K


/Users/c/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/c/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
with open('dna.txt', 'r') as myfile:
  dna = myfile.read()
dna_basepairs = dna
dna_basepairs = re.sub(r'[0-9]([0-9])?([0-9])?([0-9])?([0-9])?([0-9])?', '', dna_basepairs)
dna_basepairs = re.sub(r'\n', '', dna_basepairs)
dna_basepairs = re.sub(r'\s+', '', dna_basepairs).strip()

Create population of arrays that span 50 index values.

In [3]:
def population(data):
    sequence_one = data[:78122]
    sequence_two = data[78123:]
    
    array = np.zeros((len(sequence_one),8))
    vectorize_sequence = {
        'a' : 1,
        't' : 2,
        'g' : 3,
        'c' : 4
    }

    for base in range(len(sequence_one)):
        base_one = sequence_one[base]
        base_two = sequence_two[base]
        array[base, vectorize_sequence[base_one]] = 1
        array[base, vectorize_sequence[base_two] + 3] = 1
        
    similarity_array = np.zeros((len(array),1))
    similarity_array = np.asarray([similarity_array[i] == 1 if np.equal(array[i, :4], array[i, 4:]).all() else similarity_array[i] == 0 for i in range(len(array))])
        
    
    return [np.asarray(array).reshape(1, 78122, 8), similarity_array.reshape(1, 78122, 1)]

In [4]:
sequences = population(dna_basepairs)
comparison = sequences[0].reshape(1, 78122, 8)
similarity_array = sequences[1].reshape(1, 78122, 1)

In [5]:
print(comparison.shape)
print(similarity_array.shape)

(1, 78122, 8)
(1, 78122, 1)


looping through previous and current index of the population in order to pass each sequence through the RNN for evaluation

In [6]:
network_population = []
for i in range(30):
    model = Sequential()
    # add Long short term memory RNN and a dense layer. Compile the model
    model.add(LSTM(10, input_shape=(comparison.shape[1],8), activation='tanh', return_sequences=True, use_bias=True))
    model.add(Dense(1, activation='tanh', use_bias=True))
    network_population.append(model)

In [7]:
def grade(pop):
    mse_vec = []
    for i in network_population:
        # calculate mean squared error
        def mse(y_true, y_pred):
            y_true = y_true.reshape(1, y_true.shape[1])
            y_pred = y_pred.reshape(1, y_pred.shape[1])
            mse = mean_squared_error(y_true, y_pred)
            return mse

        y_pred = i.predict(comparison)
        mse = mse(similarity_array, y_pred)
        mse_vec.append(mse)
    mse_vec = np.asarray(mse_vec)


    # arg sort MSE and separate the fittest networks from the weakest.
    sorted_mse_idx = np.argsort(mse_vec)
    fittest_mse = sorted_mse_idx[:10]
    weakest_mse = sorted_mse_idx[10:]

    fittest_models = []
    weakest_models = []

    for model in fittest_mse:
        fittest_models.append(network_population[model])
    for model in weakest_mse:
        weakest_models.append(network_population[model])
    return [fittest_models, weakest_models]

In [8]:
def crossover(fittest, weakest):
    fittest_weights = fittest.get_weights()
    weakest_weights = weakest.get_weights()
    random_weights = []
    # randomly select index's of array to be replaced
    for layer in fittest_weights:
        random_weights.append(bernoulli.rvs(.5, size=layer.shape))
    #select weights to take from fittest weights
    select_fittest = [random_layer * network_layer for random_layer, network_layer in zip(random_weights, fittest_weights)]
    # select weakest weights to kill off
    kill_weakest = [(1 - random_layer) * network_layer for random_layer, network_layer in zip(random_weights, weakest_weights)]
    # create the new array of weights by combining the weights from the fittest models with the weights from the weakest models
    new_weights = [kill_weakest_layer + select_fittest_layer for kill_weakest_layer, select_fittest_layer in zip(select_fittest, kill_weakest)]
    weakest.set_weights(new_weights)

In [9]:
def breed(fittest, weakest):
    for weakest_model in weakest:
        #choose random model to take weights from
        fittest_model = np.random.choice(fittest)
        #crossover weights
        crossover(fittest_model, weakest_model)
    network_population = fittest + weakest
    return network_population

In [10]:
network_population = network_population
for x in range(30):
    grade_network = grade(network_population)
    breed_networks = breed(grade_network[0], grade_network[1])
    network_population = breed_networks

In [11]:
network_population[1].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 78122, 10)         760       
_________________________________________________________________
dense_21 (Dense)             (None, 78122, 1)          11        
Total params: 771
Trainable params: 771
Non-trainable params: 0
_________________________________________________________________


In [13]:
network_population[0]

In [23]:
def mse(y_true, y_pred):
        y_true = y_true.reshape(1, y_true.shape[1])
        y_pred = y_pred.reshape(1, y_pred.shape[1])
        mse = mean_squared_error(y_true, y_pred)
        return mse

y_pred = network_population[0].predict(comparison)
mse = mse(similarity_array, y_pred)

In [24]:
mse

0.15871038

In [12]:
#train, validate, test = np.split(sequences, [int(.5*len(sequences)), int(.75*len(sequences))])
#view the various lengths of the split data.
#print('Train length:', len(train), '\n', 'Test length:', len(test), '\n', 'Validation length:', len(validate))